### Step 1: Spliting the normal and abnormal audio files

In [ ]:
#import the necessary packages

from glob import glob
import ntpath
import pandas as pd
from tqdm import tqdm
from scipy.io import wavfile
from tqdm import tqdm
from scipy.io import wavfile

In [ ]:
#After putting all the audios into a single folder read them into the python file

audios="C:/Users/Admin/Desktop/Datapirates/audio training data/all audio"
audio_files=glob(audios + '/*.wav')

In [ ]:
# create an empty list and put all the audio names into the list

filename=[]
for i in audio_files:
    filename.append(ntpath.basename(i))

In [ ]:
# slelect the csv file where the audio classes are given

data=pd.read_csv("C:/Users/Admin/Desktop/Datapirates/audio training data/normal audio with extension.csv")

In [ ]:
#make a list f audios which are labeled -1 

d1=list(data['normal data(-1)'])

In [ ]:
# taking the auio files from the list d1 and saving them into another folder in the system, this will be the normal audio files  

for i in  tqdm(d1):
    rate, s = wavfile.read("C:/Users/Admin/Desktop/Datapirates/audio training data/all audio/"+i)
    wavfile.write("C:/Users/Admin/Desktop/Datapirates/audio training data/normal/"+i,rate,s)

In [ ]:
# save the audios which are labeld with 1 into another folder and this will be the abnormal audio files
data2=pd.read_csv("C:/Users/Admin/Desktop/Datapirates/audio training data/abnormal2 with wav.csv")

In [ ]:
#Create another list d2 
#Read the files in d2 from the all audio files
#Write them into a new file
d2=list(data2['abnormal(1)'])
for i in  tqdm(d2):
    rate, s = wavfile.read("C:/Users/Admin/Desktop/Datapirates/audio training data/all audio/"+i)
    wavfile.write("C:/Users/Admin/Desktop/Datapirates/audio training data/abnormal/"+i,rate,s)

### Step2: Removing Noise from the audio data

In [ ]:
# importing the necessary packages
import pydub
from pydub import AudioSegment
from glob import glob
from scipy.signal import butter,lfilter
from scipy import signal
from scipy.signal import filtfilt
import numpy as np
import librosa as lr
from scipy.signal import hilbert
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob

##### Repeat the processes from PROCEDURE1 to PROCEDURE2 for normal and abnormal data

In [ ]:
#PROCEDURE 1
#Read the data
data_dir=r"C:\Users\admin\Desktop\Datapirates\normal and abnormal\abnormal"
audio_files=glob(data_dir + '/*.wav')

In [ ]:
#PROCEDURE 2
# Read the audio files with a sample rate 4000 and append the amplitude arrays into a dictionary
signdict={}
for audio in audio_files:
    array,frq=lr.core.load(audio,sr=4000)
    signdict[audio]=array

In [ ]:
#PROCEDURE 3
# Take the amplitude values between 25khz to 400 khz sing a 4th order buttoroth bandpass filter.
# The amplitude values which exceeds these two limit points will be noise
butterout={}
for key,array in signdict.items():
    N=4
    nyq=0.5*4000
    low= 25 / nyq
    high= 400 / nyq
    b,a=signal.butter(N,[low,high],btype='band')
    outs=signal.filtfilt(b,a,array)
    output=abs(outs)
    butterout[key]=output

In [ ]:
#PROCEDURE 4
#Split the audios into 5 seconds 
#This will help you to increase your samples
start=0
end=0
interval=5000
counter=1
emplist=[]
for key,audio in butterout.items():
    n=len(audio)
    for i in range(0,n,interval):
        if i == 0:
            start=0
            end=interval
        else:
            start=end
            end=start+interval
        if end >= n:
            end=n
        chunk=audio[start:end]
        emplist.append(chunk)

In [ ]:
#PROCEDURE 5
#Find the mean values of each array 
#Repalce the values wich are three times greater than the mean value

for i in range(len(emplist)):
    mean=emplist[i].mean()
    for j in range(len(emplist[i])):
        if emplist[i][j] >= 3*mean:
            emplist[i][j]=0

In [ ]:
#PROCEDURE 6
# Perform hilbert transform on the data to rotate it 90 degree
b=[]
def hilber(i):
    c=hilbert(i)
    b.append(c)
    return b
for i in emplist:
    h=hilber(i)

In [ ]:
#PROCEDURE 7
#Get the absolute values of the amplitude from hilbrt transform 
ampl=[]
def amplitude(j):
    amp=np.abs(j)
    ampl.append(amp)
    return ampl
for j in h:
    k=amplitude(j)

In [ ]:
#PROCEDURE 8
# Make the spctrogram of the images and save it as a jpg image file
def spectrogram(i):
    plt.specgram(i,Fs=4000)
    plt.savefig(str(j+1)+".jpg")
    plt.close()
j=0
for i in k:
    j=j+1
    spectrogram(i)

In [ ]:
#PROCEDURE 9
#Use a crop function to cut the x and y axis labels 
data_dir=r"C:\Users\admin\Desktop\Datapirates\Spectrograms\Spectrogram normal"#Read the data
image=glob(data_dir + '/*.jpg')

#Defining the crop function
def crop(i):
    imageobject=Image.open(i)
    croped=imageobject.crop((54,35,390,253))
    croped.save(str(path)+".jpg")
    
#Run the crop function on the data
path=1
for i in image:
    path=path+1
    crop(i)

##### REPEAT THESE PROCESSES FOR VALIDATION DATA #####

### Spliting data into train and test

In [ ]:
#import the packages
import os
import random
import shutil

In [ ]:
#Create empty files to store the file names before and after spliting
filenames=[]
train_filenames=[]
test_filenames=[]
filenames1=[]

In [ ]:
#Read the file 
paths="C:/Users/admin/Desktop/Datapirates/Cropped Spectrograms normal and abnormal/crpped normal"

In [ ]:
# Split the files in 80:20 ratio 
for j in os.listdir(paths):
    filenames1.append(paths+"/"+j)
    filenames1.sort()  # make sure that the filenames have a fixed order before shuffling
    random.seed(230)
    random.shuffle(filenames1) # shuffles the ordering of filenames (deterministic given the chosen seed)
split_1 = int(0.8 * (len(filenames1)))

In [ ]:
##Save the test and train data into two variables
train_filenames +=(filenames1[:split_1])
test_filenames += (filenames1[split_1:])

In [ ]:
#Save test normal train normal test abnormal and train abnormal into different folders by changing the folder paths
dst_dir="C:/Users/admin/Desktop/Datapirates/Test/Normal"
for i in test_filenames:
    shutil.copy(i , dst_dir)

##### PREPROCESSING ON  SPECTROGRAM IMAGES

In [ ]:
# Import the packages which are necessary
import os
import cv2
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical

In [ ]:
#Resizing the images
def lo(image):
    image = image/255
    return image

#### Reading Train,Test and Validation images for inputing into model

In [ ]:
#Reading images from the Training dataset
tx = []
ty = []
#tz = []

dir_list="C:/Users/admin/Desktop/Datapirates/Train" 
for directory in os.listdir(dir_list):
    for filename in os.listdir(dir_list + '/' + directory):
        #tz.append(filename) 
        
        image = cv2.imread(dir_list + '/' + directory + '/' + filename)
        image = lo(image)
        image = cv2.resize(image,dsize=(32,32))
        
        
       
        
        tx.append(image)
        
        ###labelling
        if directory == 'Normal':
            ty.append(1)
        else:
            ty.append(0)  
        
xtrain=np.array(tx) 
ytrain=np.array(ty)

xtrain,ytrain=shuffle(xtrain,ytrain)

In [ ]:
xtrain.shape

In [ ]:
ytrain.shape

In [ ]:
#Reading images from the testing data set
x1 = []
y1 = []

dir_list="C:/Users/admin/Desktop/Datapirates/Test"   
for directory in os.listdir(dir_list):
    for filename in os.listdir(dir_list + '/' + directory):
        #z1.append(filename) 
        
        image = cv2.imread(dir_list + '/' + directory + '/' + filename)
        image=lo(image)
        image = cv2.resize(image,dsize=(32,32))
        
        x1.append(image)
        
        if directory == 'Normal':
            y1.append(1)
        else:
            y1.append(0) 
            

xtest=np.array(x1)
ytest=np.array(y1)
xtest,ytest=shuffle(xtest,ytest)     

In [ ]:
xtest.shape

In [ ]:
ytest.shape

In [ ]:
#Reading images from the validation dataset
val=[]
file=[]


dir_list="C:/Users/admin/Desktop/Datapirates/Validation cropped"   
for directory in os.listdir(dir_list):
    for filename in os.listdir(dir_list + '/' + directory):
        #file.append(filename) 
        
        image = cv2.imread(dir_list + '/' + directory + '/' + filename)
        image=lo(image)
        image=cv2.resize(image,dsize=(32,32))
        
        
        val.append(image)
        
        
        if directory == 'Normal':
            file.append(1)
        else:
            file.append(0)
            
xval=np.array(val)
yval=np.array(file)
xval,yval=shuffle(xval,yval)

In [ ]:
xval.shape

In [ ]:
yval.shape

In [ ]:
xtrain=np.array(tx)
ytrain=np.array(ty)
ytest=np.array(y1)
xtest=np.array(x1)

In [ ]:
ytrain1 = to_categorical(ytrain)

In [ ]:
ytrain1.shape

In [ ]:
xtrain = xtrain.astype('float32')
xtest = xtest.astype('float32')
xval = xval.astype('float32')

# Transform lables to one-hot encoding
ytrain = to_categorical(ytrain)
ytest = to_categorical(ytest)
yval=to_categorical(yval)
# Reshape the dataset into 4D array
xtrain = xtrain.reshape(xtrain.shape[0], 32,32,3)
xtest = xtest.reshape(xtest.shape[0], 32,32,3)
xval = xval.reshape(xval.shape[0], 32,32,3)